In [1]:
# =========================
# --- CELL 1: SETUP ---
# =========================
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm
import warnings

# Cài đặt thư viện tokenizers nếu chưa có
try:
    from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
except ImportError:
    !pip -q install tokenizers
    from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers

warnings.filterwarnings("ignore")

# --- Fix seed ---
def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device:", device)


✅ Device: cuda


In [ ]:


# --- CELL 2: DATA READING FUNCTION ---
def read_parallel_files(src_filename, tgt_filename):
    """Đọc cặp file song ngữ, trả về list các tuple (câu_nguồn, câu_đích)"""
    # Kiểm tra đường dẫn (hỗ trợ cả thư mục hiện tại và thư mục input của Kaggle)
    possible_paths = ["./", "/kaggle/input/", "/kaggle/working/"]
    
    src_path, tgt_path = None, None
    for p in possible_paths:
        if os.path.exists(os.path.join(p, src_filename)):
            src_path = os.path.join(p, src_filename)
        if os.path.exists(os.path.join(p, tgt_filename)):
            tgt_path = os.path.join(p, tgt_filename)
            
    if not src_path or not tgt_path:
        print(f"⚠️ Không tìm thấy file {src_filename} hoặc {tgt_filename}. Bỏ qua.")
        return []

    print(f"📖 Đang đọc: {src_path} và {tgt_path}")
    with open(src_path, 'r', encoding='utf-8') as f_src, \
         open(tgt_path, 'r', encoding='utf-8') as f_tgt:
        src_lines = [line.strip() for line in f_src.read().splitlines()]
        tgt_lines = [line.strip() for line in f_tgt.read().splitlines()]
    
    # Lọc bỏ các cặp câu rỗng hoặc lệch dòng
    pairs = []
    min_len = min(len(src_lines), len(tgt_lines))
    for i in range(min_len):
        if src_lines[i] and tgt_lines[i]:
            pairs.append((src_lines[i], tgt_lines[i]))
            
    return pairs

In [21]:
# --- CELL 3: LOAD DATA ---
print("\n--- ĐANG TẢI DỮ LIỆU ---")
# Đảm bảo tên file khớp với file bạn upload
train_pairs = read_parallel_files("/kaggle/input/maindata/train.en", "/kaggle/input/maindata/train.vi")
test_pairs = read_parallel_files("/kaggle/input/maindata/tst2013.en", "/kaggle/input/maindata/tst2013.vi")
val_pairs = read_parallel_files("/kaggle/input/maindata/tst2012.en", "/kaggle/input/maindata/tst2012.vi")
print(f"✅ Train size: {len(train_pairs)}")
print(f"✅ Test size: {len(test_pairs)}")

if len(train_pairs) > 0:
    print(f"🔎 Ví dụ mẫu: {train_pairs[0]}")


--- ĐANG TẢI DỮ LIỆU ---
📖 Đang đọc: /kaggle/input/maindata/train.en và /kaggle/input/maindata/train.vi
📖 Đang đọc: /kaggle/input/maindata/tst2013.en và /kaggle/input/maindata/tst2013.vi
📖 Đang đọc: /kaggle/input/maindata/tst2012.en và /kaggle/input/maindata/tst2012.vi
✅ Train size: 133166
✅ Test size: 1268
🔎 Ví dụ mẫu: ('Rachel Pike : The science behind a climate headline', 'Khoa học đằng sau một tiêu đề về khí hậu')


In [22]:
# --- CELL 4: TRAIN TOKENIZERS ---
print("\n--- HUẤN LUYỆN TOKENIZER ---")

def train_bpe_tokenizer(texts, vocab_size=8000):
    tokenizer = Tokenizer(models.BPE())
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
    tokenizer.decoder = decoders.ByteLevel()
    
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_size,
        special_tokens=["[PAD]", "[START]", "[END]", "[UNK]"],
        show_progress=False
    )
    tokenizer.train_from_iterator(texts, trainer=trainer)
    return tokenizer

# Gộp text để train tokenizer
all_src_text = [p[0] for p in train_pairs + val_pairs]
all_tgt_text = [p[1] for p in train_pairs + val_pairs]

if not all_src_text: # Dummy data nếu chưa load được file
    all_src_text = ["Hello world"]
    all_tgt_text = ["Xin chào"]

en_tokenizer = train_bpe_tokenizer(all_src_text, vocab_size=10000)
vi_tokenizer = train_bpe_tokenizer(all_tgt_text, vocab_size=10000)

# Lấy ID các token đặc biệt
PAD_ID = en_tokenizer.token_to_id("[PAD]")
START_ID = vi_tokenizer.token_to_id("[START]")
END_ID = vi_tokenizer.token_to_id("[END]")

print("✅ Tokenizer đã sẵn sàng.")


--- HUẤN LUYỆN TOKENIZER ---
✅ Tokenizer đã sẵn sàng.


In [23]:
# --- CELL 5: DATASET CLASS ---
class EnViDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return self.pairs[idx]

def collate_fn(batch):
    en_batch, vi_batch = zip(*batch)
    
    # Encode tiếng Anh (Source)
    en_enc = en_tokenizer.encode_batch(list(en_batch))
    en_ids = [e.ids for e in en_enc]
    
    # Encode tiếng Việt (Target) - Thêm START và END thủ công
    vi_ids = []
    for text in vi_batch:
        ids = vi_tokenizer.encode(text).ids
        vi_ids.append([START_ID] + ids + [END_ID])
    
    # Padding
    max_len_en = max([len(x) for x in en_ids])
    max_len_vi = max([len(x) for x in vi_ids])
    
    padded_en = [x + [PAD_ID] * (max_len_en - len(x)) for x in en_ids]
    padded_vi = [x + [PAD_ID] * (max_len_vi - len(x)) for x in vi_ids]
    
    return torch.tensor(padded_en), torch.tensor(padded_vi)

In [30]:
# --- CELL 6: DATALOADERS ---
BATCH_SIZE = 32

train_loader = DataLoader(EnViDataset(train_pairs), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(EnViDataset(val_pairs), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"DataLoaders created. Batch size: {BATCH_SIZE}")

DataLoaders created. Batch size: 32


In [25]:
# --- CELL 7: MODEL COMPONENTS ---
# --- Rotary Positional Embeddings ---
def rotate_half(x):
    x1, x2 = x.chunk(2, dim=-1)
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(x, cos, sin):
    return (x * cos) + (rotate_half(x) * sin)

class RotaryPositionalEncoding(nn.Module):
    def __init__(self, head_dim, max_seq_len=2048):
        super().__init__()
        inv_freq = 1.0 / (10000 ** (torch.arange(0, head_dim, 2).float() / head_dim))
        t = torch.arange(max_seq_len).float()
        freqs = torch.outer(t, inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos", emb.cos()[None, :, None, :])
        self.register_buffer("sin", emb.sin()[None, :, None, :])

    def forward(self, x, seq_len):
        return self.cos[:, :seq_len, :, :], self.sin[:, :seq_len, :, :]

# --- Feed Forward (SwiGLU) ---
class SwiGLU(nn.Module):
    def __init__(self, hidden_dim, intermediate_dim):
        super().__init__()
        self.w1 = nn.Linear(hidden_dim, intermediate_dim)
        self.w2 = nn.Linear(hidden_dim, intermediate_dim)
        self.w3 = nn.Linear(intermediate_dim, hidden_dim)

    def forward(self, x):
        return self.w3(F.silu(self.w1(x)) * self.w2(x))

# --- GQA Attention ---
class GQA(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_kv_heads, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.num_kv_heads = num_kv_heads
        self.head_dim = hidden_dim // num_heads
        self.num_groups = num_heads // num_kv_heads
        
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, num_kv_heads * self.head_dim)
        self.v_proj = nn.Linear(hidden_dim, num_kv_heads * self.head_dim)
        self.o_proj = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = dropout

    def forward(self, x, enc_out=None, mask=None, rope_cos=None, rope_sin=None):
        batch, seq_len, _ = x.shape
        kv_input = enc_out if enc_out is not None else x
        kv_seq_len = kv_input.shape[1]

        q = self.q_proj(x).view(batch, seq_len, self.num_heads, self.head_dim)
        k = self.k_proj(kv_input).view(batch, kv_seq_len, self.num_kv_heads, self.head_dim)
        v = self.v_proj(kv_input).view(batch, kv_seq_len, self.num_kv_heads, self.head_dim)

        if rope_cos is not None and enc_out is None:
            q = apply_rotary_pos_emb(q, rope_cos, rope_sin)
            k = apply_rotary_pos_emb(k, rope_cos, rope_sin)

        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        if self.num_groups > 1:
            k = k[:, :, None, :, :].expand(batch, self.num_kv_heads, self.num_groups, kv_seq_len, self.head_dim).reshape(batch, self.num_heads, kv_seq_len, self.head_dim)
            v = v[:, :, None, :, :].expand(batch, self.num_kv_heads, self.num_groups, kv_seq_len, self.head_dim).reshape(batch, self.num_heads, kv_seq_len, self.head_dim)

        out = F.scaled_dot_product_attention(q, k, v, attn_mask=mask, dropout_p=self.dropout if self.training else 0.0)
        return self.o_proj(out.transpose(1, 2).reshape(batch, seq_len, -1))

In [26]:
# --- CELL 8: TRANSFORMER MODEL ---
class TransformerBlock(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_kv_heads, dropout=0.1, is_decoder=False):
        super().__init__()
        self.norm1 = nn.RMSNorm(hidden_dim)
        self.attn = GQA(hidden_dim, num_heads, num_kv_heads, dropout)
        self.is_decoder = is_decoder
        if is_decoder:
            self.norm2 = nn.RMSNorm(hidden_dim)
            self.cross_attn = GQA(hidden_dim, num_heads, num_kv_heads, dropout)
        self.norm_ffn = nn.RMSNorm(hidden_dim)
        self.ffn = SwiGLU(hidden_dim, hidden_dim * 4)

    def forward(self, x, enc_out=None, mask=None, cross_mask=None, rope_cos=None, rope_sin=None):
        x = x + self.attn(self.norm1(x), mask=mask, rope_cos=rope_cos, rope_sin=rope_sin)
        if self.is_decoder:
            x = x + self.cross_attn(self.norm2(x), enc_out=enc_out, mask=cross_mask)
        x = x + self.ffn(self.norm_ffn(x))
        return x

class Transformer(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, hidden_dim=256, num_layers=4, num_heads=8, num_kv_heads=4):
        super().__init__()
        self.src_emb = nn.Embedding(src_vocab, hidden_dim)
        self.tgt_emb = nn.Embedding(tgt_vocab, hidden_dim)
        self.rope = RotaryPositionalEncoding(hidden_dim // num_heads)
        self.encoders = nn.ModuleList([TransformerBlock(hidden_dim, num_heads, num_kv_heads) for _ in range(num_layers)])
        self.decoders = nn.ModuleList([TransformerBlock(hidden_dim, num_heads, num_kv_heads, is_decoder=True) for _ in range(num_layers)])
        self.final_norm = nn.RMSNorm(hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, tgt_vocab)

    def forward(self, src, tgt, src_mask, tgt_mask):
        x = self.src_emb(src)
        rope_cos, rope_sin = self.rope(x, x.shape[1])
        for layer in self.encoders:
            x = layer(x, mask=src_mask, rope_cos=rope_cos, rope_sin=rope_sin)
        enc_out = x
        
        x = self.tgt_emb(tgt)
        rope_cos_tgt, rope_sin_tgt = self.rope(x, x.shape[1])
        for layer in self.decoders:
            x = layer(x, enc_out=enc_out, mask=tgt_mask, cross_mask=src_mask, rope_cos=rope_cos_tgt, rope_sin=rope_sin_tgt)
        return self.fc_out(self.final_norm(x))

In [27]:
# --- CELL 9: INIT TRAINING ---
def create_masks(src, tgt):
    src_mask = (src == PAD_ID).unsqueeze(1).unsqueeze(2).float() * -1e9
    batch, seq_len = tgt.shape
    causal = torch.triu(torch.full((seq_len, seq_len), float('-inf'), device=device), diagonal=1)
    tgt_pad = (tgt == PAD_ID).unsqueeze(1).unsqueeze(2).float() * -1e9
    return src_mask, causal + tgt_pad

model = Transformer(
    src_vocab=en_tokenizer.get_vocab_size(),
    tgt_vocab=vi_tokenizer.get_vocab_size(),
    hidden_dim=256, num_layers=4, num_heads=8, num_kv_heads=4
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID, label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.0001)

print("Model Initialized.")

Model Initialized.


In [28]:
import math
import torch

class EarlyStopping:
    def __init__(self, patience=3, min_delta=1e-4, mode="min"):
        """
        mode="min": metric càng nhỏ càng tốt (val_loss)
        """
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode

        self.best = None
        self.num_bad_epochs = 0

    def _is_improvement(self, current):
        if self.best is None:
            return True
        if self.mode == "min":
            return current < (self.best - self.min_delta)
        else:
            return current > (self.best + self.min_delta)

    def step(self, current):
        """
        return True nếu nên STOP
        """
        if self._is_improvement(current):
            self.best = current
            self.num_bad_epochs = 0
            return False
        else:
            self.num_bad_epochs += 1
            return self.num_bad_epochs >= self.patience


def save_checkpoint(path, model, optimizer, epoch, val_loss):
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "val_loss": val_loss,
    }, path)


In [35]:
# import gc, torch
# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()

In [36]:
# EPOCHS = 50
# early = EarlyStopping(patience=3, min_delta=1e-4, mode="min")
# best_path = "best_transformer_en_vi.pt"

# print("\n--- BẮT ĐẦU HUẤN LUYỆN ---")

# for epoch in range(EPOCHS):
#     model.train()
#     train_loss = 0
#     pbar = tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

#     for src, tgt in pbar:
#         src, tgt = src.to(device), tgt.to(device)
#         tgt_input, tgt_real = tgt[:, :-1], tgt[:, 1:]
#         src_mask, tgt_mask = create_masks(src, tgt_input)

#         optimizer.zero_grad()
#         output = model(src, tgt_input, src_mask, tgt_mask)
#         loss = criterion(output.reshape(-1, output.shape[-1]), tgt_real.reshape(-1))
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         optimizer.step()

#         train_loss += loss.item()
#         pbar.set_postfix(loss=f"{loss.item():.4f}")

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for src, tgt in val_loader:
#             src, tgt = src.to(device), tgt.to(device)
#             tgt_input, tgt_real = tgt[:, :-1], tgt[:, 1:]
#             src_mask, tgt_mask = create_masks(src, tgt_input)
#             output = model(src, tgt_input, src_mask, tgt_mask)
#             val_loss += criterion(output.reshape(-1, output.shape[-1]), tgt_real.reshape(-1)).item()

#     avg_train = train_loss / len(train_loader)
#     avg_val = val_loss / len(val_loader)
#     print(f"Epoch {epoch+1} | Train Loss: {avg_train:.4f} | Val Loss: {avg_val:.4f}")

#     # save best + early stopping
#     if early.best is None or avg_val < early.best - early.min_delta:
#         save_checkpoint(best_path, model, optimizer, epoch+1, avg_val)
#         print(f"  ↳ Saved BEST to {best_path} (val_loss={avg_val:.4f})")

#     if early.step(avg_val):
#         print(f"🛑 Early stopping at epoch {epoch+1}. Best val_loss = {early.best:.4f}")
#         break

# # (tuỳ chọn) vẫn lưu last state
# torch.save(model.state_dict(), "last_transformer_en_vi.pth")



--- BẮT ĐẦU HUẤN LUYỆN ---


Epoch 1/50: 100%|██████████| 4162/4162 [04:58<00:00, 13.95it/s, loss=3.8881]


Epoch 1 | Train Loss: 4.1989 | Val Loss: 3.7663
  ↳ Saved BEST to best_transformer_en_vi.pt (val_loss=3.7663)


Epoch 2/50: 100%|██████████| 4162/4162 [05:00<00:00, 13.87it/s, loss=3.4853]


Epoch 2 | Train Loss: 3.4529 | Val Loss: 3.5373
  ↳ Saved BEST to best_transformer_en_vi.pt (val_loss=3.5373)


...

Epoch 8 | Train Loss: 2.6860 | Val Loss: 3.3418


Epoch 9/50: 100%|██████████| 4162/4162 [04:59<00:00, 13.91it/s, loss=2.7402]


Epoch 9 | Train Loss: 2.6237 | Val Loss: 3.3579


Epoch 10/50: 100%|██████████| 4162/4162 [04:59<00:00, 13.91it/s, loss=2.5059]


Epoch 10 | Train Loss: 2.5656 | Val Loss: 3.3702
🛑 Early stopping at epoch 10. Best val_loss = 3.3381


In [37]:
import torch
import torch.nn.functional as F

@torch.no_grad()
def beam_search_decode(
    model,
    src_ids: torch.Tensor,          # shape: (1, src_len)
    src_mask: torch.Tensor,         # shape: (1, 1, 1, src_len) hoặc tương thích model
    beam_size: int = 5,
    max_len: int = 100,
    len_penalty: float = 0.6,       # >0 giúp tránh ưu tiên câu quá ngắn
    temperature: float = 1.0,
):
    """
    Trả về: best_hyp_ids (List[int]) gồm [START, ..., END] (có thể không có END nếu max_len).
    Ghi chú: code này phù hợp với model.forward(src, tgt, src_mask, tgt_mask) như notebook của anh.
    """
    device = src_ids.device
    model.eval()

    # mỗi beam: (token_ids, sum_logprob)
    beams = [([START_ID], 0.0)]
    finished = []

    for _ in range(max_len):
        new_beams = []

        for tok_ids, score in beams:
            # nếu đã kết thúc thì đưa vào finished
            if tok_ids[-1] == END_ID:
                finished.append((tok_ids, score))
                continue

            tgt = torch.tensor([tok_ids], device=device)  # (1, tlen)
            tlen = tgt.size(1)

            # causal mask giống CELL 11
            tgt_mask = torch.triu(
                torch.full((tlen, tlen), float('-inf'), device=device),
                diagonal=1
            )

            out = model(src_ids, tgt, src_mask, tgt_mask)  # (1, tlen, vocab)
            logits = out[0, -1, :] / max(temperature, 1e-8)  # (vocab,)

            log_probs = F.log_softmax(logits, dim=-1)       # (vocab,)
            topk_log_probs, topk_ids = torch.topk(log_probs, k=beam_size)

            for lp, wid in zip(topk_log_probs.tolist(), topk_ids.tolist()):
                new_tok_ids = tok_ids + [wid]
                new_score = score + lp
                new_beams.append((new_tok_ids, new_score))

        if not new_beams:
            break

        # length normalization
        def normed(s, length):
            # length tính theo số token sinh ra (không tính START)
            denom = ((5 + length) / 6) ** len_penalty
            return s / denom

        # giữ top beam_size
        new_beams.sort(key=lambda x: normed(x[1], max(1, len(x[0]) - 1)), reverse=True)
        beams = new_beams[:beam_size]

        # nếu tất cả beam đều END thì dừng
        if all(b[0][-1] == END_ID for b in beams):
            finished.extend(beams)
            break

    if finished:
        finished.sort(key=lambda x: x[1] / (((5 + max(1, len(x[0]) - 1)) / 6) ** len_penalty), reverse=True)
        return finished[0][0]
    return beams[0][0]


def translate_beam(sentence: str, beam_size=8, max_len=100, len_penalty=0.6):
    model.eval()
    with torch.no_grad():
        src = torch.tensor([en_tokenizer.encode(sentence).ids]).to(device)
        src_mask = (src == PAD_ID).unsqueeze(1).unsqueeze(2).float() * -1e9

        best_ids = beam_search_decode(
            model=model,
            src_ids=src,
            src_mask=src_mask,
            beam_size=beam_size,
            max_len=max_len,
            len_penalty=len_penalty,
        )

        # bỏ START, và (nếu có) cắt tại END
        if END_ID in best_ids:
            best_ids = best_ids[1:best_ids.index(END_ID)]
        else:
            best_ids = best_ids[1:]

        return vi_tokenizer.decode(best_ids)


In [38]:

for i in range(5):
    if len(test_pairs) > 0:
        idx = random.randint(0, len(test_pairs)-1)
        en_txt, vi_txt = test_pairs[idx]
        print(f"🔹 Input:  {en_txt}\n🔸 Target: {vi_txt}\n🚀 Model:  {translate_beam(en_txt)}\n{'-'*50}")

🔹 Input:  And he said that he needed those guns because of the trauma he &apos;d experienced as a young boy .
🔸 Target: Và anh ta nói rằng anh ta cần những cây súng này bởi vì những tổn thương mà anh đã trải qua trong quá khứ khi là một đứa trẻ .
🚀 Model:   Và ông nói rằng ông cần những súng đó vì những chấn thương mà ông đã trải qua khi còn là một cậu bé .
--------------------------------------------------
🔹 Input:  Am I South Korean or North Korean ?
🔸 Target: Tôi là người Nam Triều Tiên hay Bắc Triều Tiên ?
🚀 Model:   Tôi là người Nam Hàn hay Bắc Hàn hay Bắc Hàn ?
--------------------------------------------------
🔹 Input:  And so just as the womb entirely envelopes the embryo , which grows within it , the divine matrix of compassion nourishes the entire existence .
🔸 Target: Và bởi vì tử cung bao bọc hoàn toàn phôi thai đang phát triển trong lòng nó , ma trận thiêng liêng của tình thương nuôi dưỡng toàn bộ sự sống đó .
🚀 Model:   Và cũng như khi mà cơ thể đã phát triển thành phôi t

In [ ]:
# Test thủ công 1 câu riêng
custom_sentence = "I really like this model."
print(f"\n🚀 Custom Test Input: {custom_sentence}")
print(f"✅ Model Translation: {translate_beam(custom_sentence)}")

In [39]:
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.3 MB/s eta 0:00:00


In [41]:
import sacrebleu
import random
from tqdm import tqdm # Thanh hiển thị tiến độ

def calculate_bleu(data_pairs, num_samples=100):
    print(f"--- 📊 ĐANG TÍNH ĐIỂM BLEU TRÊN {num_samples} MẪU ---")
    
    # Chọn ngẫu nhiên mẫu để test (hoặc lấy hết nếu num_samples=None)
    if num_samples is not None and num_samples < len(data_pairs):
        samples = random.sample(data_pairs, num_samples)
    else:
        samples = data_pairs

    preds = [] # Các câu máy dịch
    refs = []  # Các câu đáp án chuẩn

    # Bắt đầu dịch
    for en_txt, vi_txt in tqdm(samples):
        # Dịch câu tiếng Anh
        pred_sent = translate_beam(en_txt)
        
        preds.append(pred_sent)
        refs.append(vi_txt) # Sacrebleu nhận list các string cho refs

    # Tính điểm BLEU
    # refs cần được bọc trong list vì 1 câu input có thể có nhiều câu target (ở đây ta có 1)
    bleu = sacrebleu.corpus_bleu(preds, [refs])
    
    return bleu.score

# --- CHẠY TÍNH ĐIỂM ---
# Bạn có thể tăng số lượng mẫu lên len(test_pairs) để chính xác hơn (sẽ chạy lâu hơn)
score = calculate_bleu(test_pairs, num_samples=100)

print(f"\n🌟 ĐIỂM BLEU CỦA MODEL: {score:.2f}")

--- 📊 ĐANG TÍNH ĐIỂM BLEU TRÊN 100 MẪU ---


100%|██████████| 100/100 [09:54<00:00,  5.94s/it]


🌟 ĐIỂM BLEU CỦA MODEL: 28.21


Bài toán phụ


In [ ]:
CKPT_PATH = "/kaggle/input/YOUR_CKPT_DATASET/best_transformer_en_vi.pt"

# 2) data mới train.en/train.vi nằm trong Kaggle Input dataset nào thì trỏ vào đó
DATA_DIR = "/kaggle/input/YOUR_NEW_TRAIN_DATASET"

SRC_PATH = os.path.join(DATA_DIR, "train.en")
TGT_PATH = os.path.join(DATA_DIR, "train.vi")

print("CKPT_PATH:", CKPT_PATH)
print("SRC_PATH:", SRC_PATH)
print("TGT_PATH:", TGT_PATH)

In [ ]:
# =========================
# --- CELL 3: LOAD DATA ---
# =========================
print("\n--- LOAD DATA MỚI (train.en / train.vi) ---")

all_pairs = read_parallel_data(SRC_PATH, TGT_PATH, max_lines=None)
print("Tổng số cặp câu:", len(all_pairs))

# Shuffle + split 90% train, 10% val
random.shuffle(all_pairs)
split_idx = int(0.9 * len(all_pairs))
train_pairs = all_pairs[:split_idx]
val_pairs   = all_pairs[split_idx:]

print("Train pairs:", len(train_pairs))
print("Val pairs  :", len(val_pairs))


In [ ]:
# ==========================================
# --- CELL 10: LOAD BEST CHECKPOINT + RESUME ---
# ==========================================
print("\n--- LOAD best_transformer_en_vi.pt & TRAIN TIẾP ---")

ckpt = torch.load(CKPT_PATH, map_location=device)

# ckpt đúng format từ notebook cũ:
# {"epoch":..., "model_state_dict":..., "optimizer_state_dict":..., "val_loss":...}
model.load_state_dict(ckpt["model_state_dict"])

# resume optimizer nếu anh muốn train tiếp đúng trạng thái (khuyến nghị)
if "optimizer_state_dict" in ckpt:
    optimizer.load_state_dict(ckpt["optimizer_state_dict"])
    # đảm bảo optimizer states nằm đúng device
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                state[k] = v.to(device)

start_epoch = ckpt.get("epoch", 0)
best_prev_val = ckpt.get("val_loss", None)

print(f"✅ Loaded checkpoint from: {CKPT_PATH}")
print(f"   start_epoch = {start_epoch}")
print(f"   prev_best_val_loss = {best_prev_val}")


In [ ]:
# --- CELL 1: SETUP ---
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm
import warnings

# Cài đặt thư viện tokenizers nếu chưa có
try:
    import tokenizers
except ImportError:
    os.system('pip install tokenizers')
    import tokenizers

from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers

# Cấu hình thiết bị và Random Seed
warnings.filterwarnings("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔹 Đang sử dụng thiết bị: {device}")

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)